## Bokeh Chloropleth
- Using .json file created from ESRI ABS shapefiles

In [14]:
shape_path = "/users/danielcorcoran/desktop/lga_esri_16/lga16vic_simplified/lga_vic_16_simplified.shp"

In [15]:
import fiona 
shp = fiona.open(shape_path)

In [18]:
# Extract features from shapefile
district_name = [ feat["properties"]["LGA_NAME16"] for feat in shp if feat['geometry'] is not None]
district_area = [ feat["properties"]["AREASQKM16"] for feat in shp if feat['geometry'] is not None]
lga_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]
lga_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]

In [35]:
''' Imports '''
import bokeh
from bokeh.plotting import figure, show, output_file
import numpy
from bokeh.models import HoverTool, ColumnDataSource,LogColorMapper


''' Tools '''
TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"


''' Data '''
possible_colours = ['#edf8fb','#b3cde3','#8c96c6','#8856a7','#810f7c']
possible_categories = ['very low', 'low', 'medium', 'high', 'very high']
test_colors = []
test_categories = []

for n in range(len(lga_x)):
    random_index = numpy.random.randint(0, len(possible_colours))
    test_colors.append(possible_colours[random_index])
    test_categories.append(possible_categories[random_index])
    
color_map = LogColorMapper(palette = possible_colours)

random_value1 = [numpy.random.uniform(0,10) for n in range(len(lga_x))]    
random_value2 = [numpy.random.uniform(123,549) for n in range(len(lga_x))]

source = ColumnDataSource(
    data=dict(x=lga_x, 
              y=lga_y,
             value1 = random_value1,
             value2 = random_value2,
             lganame = district_name,
             lgaarea = district_area,
             colourcode = test_colors,
             categories = test_categories))


''' Create figure '''
p = figure(
    title="Local Government Areas (2016) Bokeh Plot",
    toolbar_location="right",
    plot_width=1100,
    plot_height=800,
    tools=TOOLS)


''' Patches '''
p.patches(
    'x',
    'y',
    source=source,
    fill_alpha=1,
    fill_color = 'colourcode',
    #fill_color={'field': 'value1', 'transform': color_map},
    line_color = 'white', 
    line_width = 0.2,
legend = 'categories')


''' Remove grid lines '''
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.legend.location = "top_right"


''' Set Hover Tool '''
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [('Value 1', '@value1'),
                  ('Value 2','@value2'),
                  ('Longitude, Latitude', '$x,$y'),
                 ('LGA Name', '@lganame'),
                 ('LGA Area', '@lgaarea')]


''' Output file optional'''
#output_file("choropleth.html", title="choropleth.py example")


show(p)

In [84]:
with open('/users/danielcorcoran/desktop/lga_esri_16/LGA16_SIMPLIFIED.geojson','r') as f:
    test_data = f.read()
    
test_data2 = json.loads(test_data)

features = test_data2['features']

cols = ['#edf8fb','#b3cde3','#8c96c6','#8856a7','#810f7c']

for crap in features:
    something_random = numpy.random.randint(0, len(cols))
    crap['properties']['some_metric'] = (something_random + 1) * 12
    crap['properties']['colour_code'] = cols[something_random] 
    
test_data2['features'] = features

In [86]:
test_data2['features'][0]

{'type': 'Feature',
 'properties': {'LGA_CODE16': '20110',
  'LGA_NAME16': 'Alpine (S)',
  'STE_CODE16': '2',
  'STE_NAME16': 'Victoria',
  'AREASQKM16': 4788.1568,
  'some_metric': 48,
  'colour_code': '#8856a7'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[146.67057224400003, -36.56828108499997],
    [146.67051924600003, -36.56750909899995],
    [146.6701462520001, -36.566621097999985],
    [146.6699692640001, -36.56573409799995],
    [146.66993320200004, -36.56493308499995],
    [146.67018220500006, -36.56340108099994],
    [146.67019921400004, -36.56287207399998],
    [146.66996922700002, -36.56242809199995],
    [146.6694712210001, -36.56194109799998],
    [146.6693292000001, -36.56151208299997],
    [146.66950724700007, -36.56096809099995],
    [146.6701821680001, -36.55993806599997],
    [146.67114117000006, -36.55915105799994],
    [146.67111715300007, -36.55881805799993],
    [146.67089001300008, -36.55845299699996],
    [146.66680453100003, -36.55808832499997],
    [

In [87]:
from bokeh.io import show, output_notebook, output_file
from bokeh.models import (GeoJSONDataSource, HoverTool, LinearColorMapper)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6

with open('/users/danielcorcoran/desktop/lga_esri_16/LGA16_SIMPLIFIED.geojson',
          'r') as f:
    geo_source = GeoJSONDataSource(geojson=json.dumps(test_data2))

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(
    title="Title",
    tools=TOOLS,
    x_axis_location=None,
    y_axis_location=None,
    width=1200,
    height=800)
p.grid.grid_line_color = None

p.patches(
    'xs',
    'ys',
    fill_alpha=1,
    fill_color='colour_code',
    line_color='white',
    line_width=0.5,
    source=geo_source)

hover = p.select_one(HoverTool)
# hover.point_policy = "follow_mouse"
hover.tooltips = [("LGA NAME", "@LGA_NAME16"), ("LGA CODE", "@LGA_CODE16"),
                  ("LGA SQKM", "@AREASQKM16"), ("SOME METRIC", "@some_metric")]

show(p)